In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import glob
import torch
import matplotlib.pyplot as plt
import pandas as pd
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
x = np.load("/app/dataset/train_numpy_16k/ee6183c02ec9.npy")

In [23]:
x.mean()

-4.1994823e-05

In [2]:
traindf = pd.read_csv("/app/dataset/valid_data.csv")
traindf = traindf.sample(1000).reset_index(drop=True)


In [3]:
files = traindf.id.apply(lambda x: os.path.join("/app/dataset/train_numpy_16k",x.replace(".mp3",".npy")))

In [4]:
from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from configs.wav2vec2_characterwise_pretrained_ctc_augs import Configs
# CFG = Configs(files,["" for _ in files])
CFG = Configs(files,traindf.sentence,use_numpy=True)

inference mode is on


In [6]:
data_loader = DataLoader(CFG.inference_dataset,batch_size=16, pin_memory=True, num_workers=2,shuffle=False,collate_fn=CFG.dataloder_collate)

In [24]:
device = "cuda"
CFG.load_state_dict("/app/bengali-speech-recognition/workdir/wav2vec2_characterlevel_pretrained_ctcloss_noaugs/bestmodel_wer.pkl")
CFG.model.to(device)
CFG.model.eval()
1

loading model checkpoint from epoch:  20000


1

In [25]:
def infer(inputs):
    all_indices = torch.argmax(CFG.model(inputs).detach().cpu(), dim=-1)
    generated = []
    for indices in all_indices:
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = indices[indices != CFG.BLANK_TOKEN]
        generated.append(indices)
    return generated

In [26]:
predictions = []
with torch.no_grad():
    for batch in tqdm(data_loader):
        inputs= batch[0]
        generated_tokens = infer(inputs.to(device))
        for gen in generated_tokens:
            try:
                hypothesis = CFG.tokenizer.decode_torch_inference(gen)
            except:
                predictions.append("")
            predictions.append(hypothesis)

100%|██████████| 63/63 [00:15<00:00,  3.98it/s]


In [27]:
def check(sentence):
    if len(sentence)==0:
        return '।'
    return sentence

In [28]:
df = pd.DataFrame({"id":files,"sentence":predictions})
df.id = df.id.apply(lambda x: os.path.split(x)[-1].split(".")[0])
df.sentence= df.sentence.apply(lambda x:normalize(x))
df.sentence = df.sentence.apply(lambda x:check(x))

In [29]:
from jiwer import wer, cer

In [30]:
wers = [wer(a,b) for a,b in zip(traindf.sentence,df.sentence)]

In [31]:
import numpy as np
np.mean(wers)

0.4328951264207069

In [18]:
traindf

,id,sentence,split
0,ee6183c02ec9.mp3,ফলে এক বছর পরই মক্কা থেকে তিন হাজার সুসজ্জিত স...,valid
1,96f993454bea.mp3,এই ওষুধ সাধারণত এক ঘণ্টার মধ্যে কাজ করা শুরু করে।,valid
2,57d43532d3ba.mp3,রেডিওতে আরও বলা হয়েছে যে ইহুদিদের নির্মূল করা...,valid
3,0f814ac9a6f3.mp3,এটি প্রাথমিক শিক্ষা অধিদপ্তর এর অধীনে পরিচালিত।,valid
4,c3f7599d3f79.mp3,তার শখ ছবি আঁকা।,valid
...,...,...,...
995,274e77aa019b.mp3,নাইজারের দক্ষিণাংশে অধিকাংশ মানুষ বসবাস করে।,valid
996,ebf86f865251.mp3,এটিই 'শ্যামসুন্দর মন্দির' নামে পরিচিত।,valid
997,ee0e587bf481.mp3,আশেপাশের বিভিন্ন এলাকা থেকে অনেক খুচরা ও পাইকা...,valid
998,93dbaaa14ad0.mp3,এরপরের দুই টেস্ট খেলার জন্যে তাকে দলে রাখা হয়নি।,valid


In [17]:
df

,id,sentence
0,ee6183c02ec9,ফলে একবছর পরেই মুকাথে তিন হাজার সুসযজিত সন্যে ...
1,96f993454bea,এ অষুর সাঘারণতে এক ঘনটার মে কাছে বারা শুরু করে
2,57d43532d3ba,রেডিওতে আরো বলা হয়েছিলে ইহদিগে নির্ম করা আহবাঞ...
3,0f814ac9a6f3,এটি প্রাথমি শিক অধিদপ্তরেরর অধিনে পরিজালিত
4,c3f7599d3f79,তার শখছবি আকা
...,...,...
995,274e77aa019b,নাইজারের দক্ষিণাংশে অধিকাংশ মানুষ বসবাস করে
996,ebf86f865251,এটি সযামসুন্দর মন্দির নামে পরিচিত
997,ee0e587bf481,আশীপাশের বিভিন্ন এলাকা থেকে অনেক খুঁচরা ও বাইক...
998,93dbaaa14ad0,এরপরের দুই টেস্টখেলার জন্যে তাকে দলে রাখা হয়নি
